# Escalado

In [1]:
import pickle

In [2]:
import numpy as np

import pandas as pd

% matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
# sns.set()

from sklearn.preprocessing import StandardScaler

In [3]:
s2_5k = pd.concat([
    pd.read_pickle("data/o3o4vZ/s2_5k.pkl.bz2"),
    pd.read_pickle("data/paper/news_s2500.pkl.bz2")], ignore_index=True)

s5k = pd.concat([
    pd.read_pickle("data/o3o4vZ/s5k.pkl.bz2"),
    pd.read_pickle("data/paper/news_s5k.pkl.bz2")], ignore_index=True)

s20k = pd.concat([
    pd.read_pickle("data/o3o4vZ/s20k.pkl.bz2"),
    pd.read_pickle("data/paper/news_s20k.pkl.bz2")], ignore_index=True)

sO2O = pd.read_pickle("data/paper/sO2O.pkl.bz2")

Agregamos la columna tile

In [4]:
def add_tile(df):
    df = df.copy()
    df["tile"] = df.id.apply(lambda i: "b" + str(i)[1:4])
    return df

s2_5k = add_tile(s2_5k)
s5k = add_tile(s5k)
s20k = add_tile(s20k)
sO2O = add_tile(sO2O)

In [5]:
print sorted(s5k.tile.unique())
print sorted(s2_5k.tile.unique())
print sorted(s20k.tile.unique())
print sorted(sO2O.tile.unique())

['b206', 'b214', 'b216', 'b220', 'b228', 'b234', 'b247', 'b248', 'b261', 'b262', 'b263', 'b264', 'b277', 'b278', 'b360', 'b396']
['b206', 'b214', 'b216', 'b220', 'b228', 'b234', 'b247', 'b248', 'b261', 'b262', 'b263', 'b264', 'b277', 'b278', 'b360', 'b396']
['b206', 'b214', 'b216', 'b220', 'b228', 'b234', 'b247', 'b248', 'b261', 'b262', 'b263', 'b264', 'b277', 'b278', 'b360', 'b396']
['b206', 'b214', 'b216', 'b220', 'b228', 'b234', 'b247', 'b248', 'b261', 'b262', 'b263', 'b264', 'b277', 'b278', 'b360', 'b396']


## Removemos todo lo que no sea RRLyrae o desconocido

In [6]:
def filter_only_rr_unk(df):
    flt = (df.vs_type == "") | df.vs_type.str.startswith('RRLyr-')
    return df[flt]

s2_5k = filter_only_rr_unk(s2_5k)
s5k = filter_only_rr_unk(s5k)
s20k = filter_only_rr_unk(s20k)
sO2O = filter_only_rr_unk(sO2O)

## Removemos los features que no queremos


Removemos filas que tengan un nan en `period_fit` pero antes nos fijamos que ninguna sea una RR-Lyrae

In [7]:
print s20k[s20k.Period_fit.isnull()].vs_type.unique()
print s5k[s5k.Period_fit.isnull()].vs_type.unique()
print s2_5k[s2_5k.Period_fit.isnull()].vs_type.unique()
print sO2O[sO2O.Period_fit.isnull()].vs_type.unique()

['']
['']
['']
['']


In [8]:
s20k = s20k.dropna()
s5k = s5k.dropna()
s2_5k = s2_5k.dropna()
sO2O = sO2O.dropna()

Evaluamos que columnas tienen un valor infinito

In [9]:
for x in s20k.columns:
    if s20k[x].dtype == object:
        continue
    if np.isinf(s20k[x].values).sum() + np.isinf(s5k[x].values).sum() + np.isinf(s2_5k[x].values).sum() + np.isinf(sO2O[x].values).sum():
        print x

Period_fit


Como period_fit es un feature que me interesa, verificamos que cantidad de filas son las afectadas
y si hay alguna con RRLyraes

In [10]:
print s2_5k[np.isinf(s2_5k.Period_fit.values)].shape
print s5k[np.isinf(s5k.Period_fit.values)].shape
print s20k[np.isinf(s20k.Period_fit.values)].shape
print sO2O[np.isinf(sO2O.Period_fit.values)].shape

print s2_5k[np.isinf(s2_5k.Period_fit.values)].vs_type.unique()
print s5k[np.isinf(s5k.Period_fit.values)].vs_type.unique()
print s20k[np.isinf(s20k.Period_fit.values)].vs_type.unique()
print sO2O[np.isinf(sO2O.Period_fit.values)].vs_type.unique()

(7, 85)
(10, 85)
(51, 85)
(2, 85)
['']
['']
['']
['']


Son pocas filas y no hay rrlyraes... las borramos

In [11]:
s20k = s20k[~np.isinf(s20k.Period_fit.values)]
s5k = s5k[~np.isinf(s5k.Period_fit.values)]
s2_5k = s2_5k[~np.isinf(s2_5k.Period_fit.values)]
sO2O = sO2O[~np.isinf(sO2O.Period_fit.values)]

ahora almacenamos tod esto limpio para futuros usos

In [18]:
s20k.to_pickle("data/paper/nonull/s20k.pkl.bz2", compression="bz2")
s5k.to_pickle("data/paper/nonull/s5k.pkl.bz2", compression="bz2")
s2_5k.to_pickle("data/paper/nonull/s2_5k.pkl.bz2", compression="bz2")
sO2O.to_pickle("data/paper/nonull/sO2O.pkl.bz2", compression="bz2")

## Ahora normalizamos

In [24]:
no_features = ["id", "vs_catalog", "vs_type", "ra_k", "dec_k", "tile"] 
X_columns = [c for c in s20k.columns if c not in no_features]

In [26]:
scaler_20k = StandardScaler()
norm_s20k = s20k.copy()
norm_s20k[X_columns] = scaler_20k.fit_transform(s20k[X_columns])
pickle.dump(scaler_20k, open("data/paper/scalers/scaler_20k.pkl", "wb"))
norm_s20k.to_pickle("data/paper/scaled/s20k.pkl.bz2", compression="bz2")

In [27]:
scaler_5k = StandardScaler()
norm_s5k = s5k.copy()
norm_s5k[X_columns] = scaler_5k.fit_transform(s5k[X_columns])
pickle.dump(scaler_5k, open("data/paper/scalers/scaler_5k.pkl", "wb"))
norm_s5k.to_pickle("data/paper/scaled/s5k.pkl.bz2", compression="bz2")

In [28]:
scaler_2_5k = StandardScaler()
norm_s2_5k = s2_5k.copy()
norm_s2_5k[X_columns] = scaler_2_5k.fit_transform(s2_5k[X_columns])
pickle.dump(scaler_2_5k, open("data/paper/scalers/scaler_2_5k.pkl", "wb"))
norm_s2_5k.to_pickle("data/paper/scaled/s2_5k.pkl.bz2", compression="bz2")

In [29]:
scaler_sO2O = StandardScaler()
norm_sO2O = sO2O.copy()
norm_sO2O[X_columns] = scaler_sO2O.fit_transform(sO2O[X_columns])
pickle.dump(scaler_sO2O, open("data/paper/scalers/scaler_sO2O.pkl", "wb"))
norm_sO2O.to_pickle("data/paper/scaled/sO2O.pkl.bz2", compression="bz2")

In [33]:
cosos = sorted(norm_s2_5k.tile.unique())
rows = {k: {"Tile": k} for k in cosos}
    
for nombre, s in zip(["UaU", "Chica", "Mediana", "Grande", ] , [sO2O, s2_5k,s5k,s20k, ]):
    s = s.copy()
    s["tile"] = s["id"].apply(lambda i: "b" + str(i)[1:4])
    grouped = s.groupby("tile")
    data = {k: grouped.get_group(k).copy() for k in grouped.groups.keys()}
    for tile, df in data.items():
        row = rows[tile]
        row.update({
            nombre : len(df[df.vs_type.str.contains("")]),
            "RR-Lyrae": len(df[df.vs_type.str.startswith("RRLyr")]),
        })
        



In [36]:
print pd.DataFrame(
    [v for k, v in sorted(rows.items())]
)[
    "Tile UaU Chica Mediana Grande RR-Lyrae".split()
].sort_values("Tile").to_latex(index=False)

\begin{tabular}{lrrrrr}
\toprule
 Tile &   UaU &  Chica &  Mediana &  Grande &  RR-Lyrae \\
\midrule
 b206 &    94 &   2547 &     5047 &   20045 &        47 \\
 b214 &    68 &   2534 &     5034 &   20031 &        34 \\
 b216 &    86 &   2540 &     5039 &   20033 &        43 \\
 b220 &   130 &   2565 &     5065 &   20065 &        65 \\
 b228 &    56 &   2528 &     5028 &   20023 &        28 \\
 b234 &   252 &   2625 &     5126 &   20124 &       126 \\
 b247 &   384 &   2692 &     5190 &   20182 &       192 \\
 b248 &   442 &   2719 &     5219 &   20197 &       222 \\
 b261 &   506 &   2751 &     5245 &   20224 &       253 \\
 b262 &   635 &   2809 &     5313 &   20267 &       318 \\
 b263 &   638 &   2819 &     5317 &   20310 &       319 \\
 b264 &   624 &   2811 &     5309 &   20300 &       312 \\
 b277 &   867 &   2923 &     5413 &   20371 &       434 \\
 b278 &   879 &   2929 &     5424 &   20363 &       440 \\
 b360 &  1364 &   3181 &     5680 &   20676 &       682 \\
 b396 &    30 